In [6]:
import csv
import pandas as pd

def csv_to_triples(csv_file, relationship_mapping, output_csv, combined_column_mappings):
    triples = []
    
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader) 
        
        for row in reader:
            subject = row[0].strip()  #First column as subject
            
            for i in range(1, len(row)):
                column_name = headers[i].strip()
                if column_name in relationship_mapping:  #Skip columns not in mapping
                    relationship = relationship_mapping[column_name]
                    obj = row[i].strip()
                    if obj:
                        triples.append((subject, relationship, obj))
            
            for combined_columns, combined_relationship in combined_column_mappings:

                combined_values = [str(row[headers.index(col)].strip()) for col in combined_columns if row[headers.index(col)].strip()]
                
                if combined_values:
                    combined_obj = ', '.join(combined_values)
                
                    #Check if it's the "is_open_from" relationship and all values are "?"
                    if combined_relationship == "is_open_from" and combined_obj == "?, ?, ?, ?, ?, ?, ?":
                        combined_obj = "We are unsure when they are open"
                
                    triples.append((subject, combined_relationship, combined_obj))
    
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Subject", "Relationship", "Object"]) 
        writer.writerows(triples)
    
    return triples

#Custom relationships
relationship_mapping = {
    "Main_Services": "main_services_are",
    "Other_Services": "other_services_are",
    "Phone_Number": "phone_number_is",
    "Website": "website_is",
    "Location_Address": "is_located_at",
    "Location_URL_Map": "url_is_located_at",
    "Availability": "has_an_availability_status_of",
    "Description": "description_is",
    "Languages": "speaks",
    "Cost": "costs",
    "Google_Review": "has_Google_Reviews",
    "Coverage": "covers",
    "Zipcode": "zipcode_is",
    "24hour": "24hours_status_is"
}

#Multiple column combinations and their respective relationships
combined_column_mappings = [
    (["Serving", "Eligibility"], "serves"),
    (["Facebook_URL", "Twitter_URL"], "socials_are"),
    (["Latitude", "Longitude"], "coordinates_are"),
    (["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], "is_open_from")
]

csv_file_path = r"C:\Users\wish0\Homeless_Services_Provider_Interface\Chenzi_Philadelphia_Temp_Shelter(19131-19154).csv" #Change to your path
output_csv = "Philadelphia_temp_shelter_triples_19131_19154.csv"  #Change to yours
triples = csv_to_triples(csv_file_path, relationship_mapping, output_csv, combined_column_mappings)

df = pd.read_csv("Philadelphia_temp_shelter_triples_19131_19154.csv") #Make sure matches the output_csv

df.to_csv("Philadelphia_temp_shelter_triples_19131_19154.csv", index=False, encoding='utf-8')

df

,Subject,Relationship,Object
0,Drop In Center (Prevention Point Philadelphia),main_services_are,"['weather relief', 'meals', 'addiction & recov..."
1,Drop In Center (Prevention Point Philadelphia),phone_number_is,215-634-5272
2,Drop In Center (Prevention Point Philadelphia),is_located_at,"2915 Kensington Avenue, Philadelphia, PA 19134"
3,Drop In Center (Prevention Point Philadelphia),url_is_located_at,https://www.google.com/maps/?q=2915+Kensington...
4,Drop In Center (Prevention Point Philadelphia),has_an_availability_status_of,available
...,...,...,...
296,Emergency Food & Shelter Services (PathStone),covers,This program covers residents of the following...
297,Emergency Food & Shelter Services (PathStone),24hours_status_is,FALSE
298,Emergency Food & Shelter Services (PathStone),serves,"['all ages', 'immigrants', 'refugees', 'undocu..."
299,Emergency Food & Shelter Services (PathStone),socials_are,"https://www.facebook.com/pathstonecorp/, https..."
